In [23]:
from __future__ import print_function

#Loading Data
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
from itertools import cycle

#To build ML model
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [24]:
def read_XYZformat(fln):
    #Modified from https://github.com/pele-python/pele/blob/master/pele/utils/xyz.py
    fin = open(fln,'r')
    natoms = int(fin.readline())
    comment = fin.readline()[:-1]
    coords = np.zeros([natoms, 3], dtype="float64")
    atomtypes = []
    for x in coords:
        line = fin.readline().split()
        atomtypes.append(line[0])
        x[:] = list(map(float, line[1:4]))
    fin.close()
    return (coords, comment, atomtypes)

In [25]:
def write_xyz(fln, coords, title="", atomtypes=("A",)):
    fout = open(fln,'w')
    fout.write("%d\n%s\n" % (coords.size / 3, title))
    for x, atomtype in zip(coords.reshape(-1, 3), cycle(atomtypes)):
        fout.write("%s %.18g %.18g %.18g\n" % (atomtype, x[0], x[1], x[2]))
    fout.close()
 

In [26]:
def center_solute(coords,com_selection):
    centered_coords = coords - com_selection.mean(axis=0)
    return centered_coords

def wrap_cell(coords,metadata,cellparam_np):
#def wrap_cell(coords,box_length):    
    #cellparam_np = np.array((box_length,box_length,box_length))
    
    wrapped_traj = coords + (metadata["box_length"]*(0.5) < np.abs(coords))*cellparam_np*(np.sign(cellparam_np*(0.5)-coords))

    return wrapped_traj

In [27]:
def reorient_molecule(newX,newY,newZ):
    #Confirm that new coordinate vectors are unit vectors
    newX = (1.0/np.linalg.norm(newX)*newX)
    newY = (1.0/np.linalg.norm(newY)*newY)
    newZ = (1.0/np.linalg.norm(newZ)*newZ)
    
    newZ_xy = np.linalg.norm(newZ[:2])
    
    precession_alpha  =  np.arctan2(newY[0]*newZ[1]-newY[1]*newZ[0],newX[0]*newZ[1]-newX[1]*newZ[0])
    nutation_beta     =  np.arctan2(newZ_xy,newZ[2])
    rotation_gamma    =  (-1.0)*np.arctan2((-1.0)*newZ[0],newZ[1])
    
    return precession_alpha, nutation_beta, rotation_gamma

In [28]:
def genF_bond(coords,topology):
    bond_lengths = np.zeros((len(topology),1))
    
    for bond in range(len(topology)):
        pair_atoms = coords[topology[bond]]
        bond_lengths[bond] = np.linalg.norm(pair_atoms[0]-pair_atoms[1])
    
    return bond_lengths

def genF_angles(coords,topology):
    def compute_angle(coords,center_pt,end_pts):
        ba = coords[end_pts[0]] - coords[center_pt[0]]
        bc = coords[end_pts[1]] - coords[center_pt[0]]
    
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(cosine_angle)
            
        return np.degrees(angle)
    
    angles_and_centers = []
    
    for pair1 in range(len(topology)-1):
        for pair2 in range(pair1+1,len(topology)):
            result_list = list(topology[pair1])
            result_list.extend(x for x in topology[pair2] if x not in result_list )
            if(len(result_list)==3):
                angles_and_centers.append((list(set(topology[pair1]).intersection(topology[pair2])),
                                           list(set(topology[pair1]).symmetric_difference(topology[pair2]))))
    
    bond_angles = np.zeros((len(angles_and_centers),1))
    for i,three_atoms in enumerate(angles_and_centers):
        bond_angles[i] = compute_angle(coords,three_atoms[0],three_atoms[1])  
    
    return bond_angles

def genF_dihedrals(coords,topology):

####-------------------------------------------   
    def compute_dihedral(coords, dihedrals_and_centers):
        
        u1,u2,u3,u4 = coords[dihedrals_and_centers]
        #print(u1,u2,u3,u4)

        a1 = u2 - u1
        a2 = u3 - u2
        a3 = u4 - u3

        v1 = np.cross(a1, a2)
        v1 = v1 / (v1 * v1).sum(-1)**0.5
        v2 = np.cross(a2, a3)
        v2 = v2 / (v2 * v2).sum(-1)**0.5
        porm = np.sign((v1 * a3).sum(-1))
        rad = np.arccos((v1*v2).sum(-1) / ((v1**2).sum(-1) * (v2**2).sum(-1))**0.5)
        if not porm == 0:
            rad = rad * porm

        return np.degrees(rad)
 ####-------------------------------------------   
    
    dihedrals_and_centers = []

    for pair1 in range(len(topology)-2):
        for pair2 in range(pair1+1,len(topology)-1):
            for pair3 in range(pair2+1,len(topology)):
                result_list = list(topology[pair1])
                result_list.extend(x for x in topology[pair2] if x not in result_list )
                result_list.extend(x for x in topology[pair3] if x not in result_list[2:] )
                if(len(result_list)==4):
                    dihedrals_and_centers.append(result_list)

    for four_atoms in dihedrals_and_centers:
        for i in topology:
            if [four_atoms[0],four_atoms[2]] == i:
                four_atoms[0],four_atoms[1] = four_atoms[1], four_atoms[0]

    bond_dihedrals= np.zeros((len(dihedrals_and_centers),1))
    for i,four_atoms in enumerate(dihedrals_and_centers):
        bond_dihedrals[i] = compute_dihedral(coords,four_atoms)     
    
    return bond_dihedrals

In [29]:
def generate_solvent_COMs(coords,metadata):
    
    #Definitions of numbers of atoms
    n_atoms_solute = metadata["n_atoms_solute"]; total_atoms = metadata["n_atoms"];
    n_atoms_solvent = len(metadata["solvent_indices"]);
    
    num_solvent = int((total_atoms-n_atoms_solute)/n_atoms_solvent)
    
    #Generating a new numpy array for the coordinates with reconstructed solvent
    reconstructed_solvent_POS = np.zeros_like(coords)
    reconstructed_solvent_POS[:n_atoms_solute,:] = coords[:n_atoms_solute,:]
    
    #Reconstruct solvent at boundaries and compute COMs
    solvent_COMs    = np.zeros((num_solvent,3))
    solvent_COM_dist = np.zeros((num_solvent,1)) 
    
    for solvent_mol in range(num_solvent):
        #Grab POS for all atoms in solvent molecule
        solvent_mol_coords = coords[n_atoms_solute+solvent_mol*n_atoms_solvent:
                                    n_atoms_solute+(solvent_mol+1)*n_atoms_solvent]
        
        bool_solvent_mol_coords = (np.abs(solvent_mol_coords - solvent_mol_coords[0,:]) > 0.5*metadata["box_length"])

        solvent_mol_coords = solvent_mol_coords+bool_solvent_mol_coords*np.sign(solvent_mol_coords[0,:])*metadata["box_length"]
        reconstructed_solvent_POS[n_atoms_solute+solvent_mol*n_atoms_solvent:
                                  n_atoms_solute+(solvent_mol+1)*n_atoms_solvent] = solvent_mol_coords

        solvent_COMs[solvent_mol]      = np.average(solvent_mol_coords,axis=0)
        solvent_COM_dist[solvent_mol]  = np.linalg.norm(np.average(solvent_mol_coords,axis=0))
        
    return reconstructed_solvent_POS, solvent_COMs, solvent_COM_dist

In [30]:
def compute_netForce_perSolvent(forces,metadata):
    #Definitions of numbers of atoms
    n_atoms_solute = metadata["n_atoms_solute"]; total_atoms = metadata["n_atoms"];
    n_atoms_solvent = len(metadata["solvent_indices"]);
    
    num_solvent = int((total_atoms-n_atoms_solute)/n_atoms_solvent)
    
    #Array to store X,Y,Z components of force
    force_per_solvent    = np.zeros((num_solvent,3))
    
    for solvent_mol in range(num_solvent):
        #Grab forces for all atoms in solvent molecule
        solvent_mol_forces = forces[n_atoms_solute+solvent_mol*n_atoms_solvent:
                                    n_atoms_solute+(solvent_mol+1)*n_atoms_solvent]
        force_per_solvent[solvent_mol] = np.sum(solvent_mol_forces,axis=0)
        
    return force_per_solvent


In [31]:
def compute_solvent_orientation(coords,metadata,solvent_COM,nearest_15_solv_index):
    
    #solvent_COM[nearest_15_solv_index]
    

    
    solvent_angles = np.zeros((len(nearest_15_solv_index),1))
    
    for i in range(len(nearest_15_solv_index)):
        a1 =  (-1.0)*solvent_COM[nearest_15_solv_index][i]
        a2 =  coords[(metadata["n_atoms_solute"]+len(metadata["solvent_indices"])*nearest_15_solv_index)+metadata["solvent_indices"]["N"]][i]-solvent_COM[nearest_15_solv_index][i]
        
        cosine_angle = np.dot(a1, a2) / (np.linalg.norm(a1) * np.linalg.norm(a2))
        solvent_angles[i] = np.arccos(cosine_angle)
            
        #return np.degrees(angle)
    
    return np.degrees(solvent_angles)


In [32]:
def align_solute(coords,x_reference_atom,y_reference_atom):
    # Here, the x_reference_atom will be aligned to the x-axis. The y_reference_atom serves to define the
    # XY-plane, but as it is not guaranteed to be orthogonal to the new x-axis, the new_Y is the orthognal direction
    # which falls in this XY-plane. the new Z-axis is simply the cross-product of these two new vectors.
    #
    # Given the new set of axis, the Euler angles can be computed in order to construct a rotation matrix used to
    # transform all coordinates from the local coordinate system into the world coordinate system.
    
    #Define new X axis
    newX = x_reference_atom
    newX = (1.0/np.linalg.norm(newX))*newX
    #Use a second point to define the plane.
    #I.e. find a vector n orthogonal to a that is in the plane which contains a and b.
    refY = y_reference_atom
    newY = np.cross(newX,np.cross(newX,(1.0/np.linalg.norm(refY))*refY))
    newY = (1.0/np.linalg.norm(newY))*newY

    #Define new Z as perpendicular to plane defined by newX and newY
    newZ = np.cross(newX,newY)
    newZ = (1.0/np.linalg.norm(newZ)*newZ)

    #Apply reoriented molecules
    alpha,beta,gamma = reorient_molecule(newX,newY,newZ)

    #Define Rotation Matrix (Z1X2Z3, Euler Angles)

    RotMatrix = np.array([[np.cos(alpha)*np.cos(gamma)-np.cos(beta)*np.sin(alpha)*np.sin(gamma),-np.cos(alpha)*np.sin(gamma)-np.cos(beta)*np.cos(gamma)*np.sin(alpha),np.sin(alpha)*np.sin(beta)],
                          [np.cos(gamma)*np.sin(alpha)+np.cos(alpha)*np.cos(beta)*np.sin(gamma),np.cos(alpha)*np.cos(beta)*np.cos(gamma)-np.sin(alpha)*np.sin(gamma),-np.cos(alpha)*np.sin(beta)],
                          [np.sin(beta)*np.sin(gamma),np.cos(gamma)*np.sin(beta),np.cos(beta)]])

    POS_rotated = np.matmul(RotMatrix,np.transpose(coords)).transpose()
    
    return POS_rotated, RotMatrix


In [33]:
def generate_POS_for_nearest(coords,metadata,num_nearest,AtomNames):

    nearest_solv_index = metadata["Solvent_COM_Indices"][:num_nearest]

    solute_and_nearest = np.zeros(((metadata["n_atoms_solute"]+len(metadata["solvent_indices"])*num_nearest),3))
    solute_and_nearest[:metadata["n_atoms_solute"],:] = coords[:metadata["n_atoms_solute"],:]
    
    for i, solv_index in enumerate(nearest_solv_index):
        solute_and_nearest[metadata["n_atoms_solute"]+i*len(metadata["solvent_indices"]):
                           metadata["n_atoms_solute"]+(i+1)*len(metadata["solvent_indices"]),:] = coords[metadata["n_atoms_solute"]+solv_index*len(metadata["solvent_indices"]):
                           metadata["n_atoms_solute"]+(solv_index+1)*len(metadata["solvent_indices"]),:]

    #solute_and_nearest = np.matmul(metadata["RotationMatrix"],np.transpose(solute_and_nearest)).transpose()
    
    write_xyz("AQ_"+str(num_nearest)+"nearestACN.xyz", solute_and_nearest, title="", atomtypes=AtomNames)
    
    return solute_and_nearest

In [34]:
def cartesian_to_spherical(coords):
    
    spherical_coords = np.zeros_like(coords)
    
    spherical_coords[:,0]   = np.linalg.norm(coords,axis=1) #rho
    spherical_coords[:,1]   = np.arccos(coords[:,2]/np.linalg.norm(coords,axis=1)) #theta
    spherical_coords[:,2]   = np.arctan2(coords[:,1],coords[:,0]) #phi
    
    return spherical_coords

def spherical_to_cartesian(coords):
    
    cartesian_coords = np.zeros_like(coords)
    
    cartesian_coords[:,0]   = coords[:,0]*np.sin(coords[:,1])*np.cos(coords[:,2]) #x
    cartesian_coords[:,1]   = coords[:,0]*np.sin(coords[:,1])*np.sin(coords[:,2]) #y
    cartesian_coords[:,2]   = coords[:,0]*np.cos(coords[:,1]) #z
    
    return cartesian_coords

## Main

In [35]:
metadata = {
    "topology_solute"       : [[4,5],[4,22],[22,15],[15,23],[15,16],[16,19],[16,18],
                               [18,21],[18,17],[17,20],[17,3],[3,22],[3,6],[6,7],
                               [6,1],[1,12],[12,14],[12,10],[10,13],[10,0],[0,11],
                               [0,8],[8,9],[8,2],[2,1],[2,4]],
    
    "n_atoms"               : 594,
    "n_atoms_solute"        : 24,
    "box_length"            : 20.55,
    
    "solvent_indices"       : {"CC": 0,
                               "C" : 1,
                               "H1": 2,
                               "H2": 3,
                               "H3": 4,
                               "N" : 5
                              }
        
}


#Create a representation of topology with reversed indices
topology_reverse = []
for i in metadata["topology_solute"]:
    topology_reverse.append(list(i))
for i in topology_reverse:
    i.reverse()
metadata["topology_reverse"] = topology_reverse

translation_vectors = np.array([[metadata["box_length"],metadata["box_length"],metadata["box_length"]]])

In [36]:
def load_and_analyze_trajstep(POS_fln,FRC_fln):
    #Load Data
    POS, _, AtomNames = read_XYZformat(str(POS_fln))
    FRC, _, AtomNames = read_XYZformat(str(FRC_fln))

    #Center and Wrap all solvent molecules. Only necessary for POS as criteria are only 
    #based on POS and the indices do not change
    cPOS = center_solute(POS,POS[:metadata["n_atoms_solute"]])  #initial translation of entire cell. COM may be incorrect if atoms need to be wrapped
    #cwPOS = wrap_cell(cPOS,metadata["box_length"])
    cwPOS = wrap_cell(cPOS,metadata,translation_vectors)
    cwPOS = center_solute(cwPOS,cwPOS[:metadata["n_atoms_solute"]]) #second re-centering after all atoms are wrapped

    #Features for positions: SOLUTE
    gF_bonds = genF_bond(cwPOS,metadata["topology_solute"])
    gF_angles = genF_angles(cwPOS,metadata["topology_solute"])
    gF_dihedrals = genF_dihedrals(cwPOS,metadata["topology_solute"])
    #Features for positions: SOLVENT
    _, solvent_COM, solvent_COM_dist = generate_solvent_COMs(cwPOS,metadata)
    metadata["Solvent_COM_Indices"] = np.argsort(solvent_COM_dist, axis=0).flatten()
    nearest_15_solv_index = np.argsort(solvent_COM_dist,axis=0)[:15,:].flatten()

    gF_solvent_distance   = solvent_COM_dist[nearest_15_solv_index]
    gF_solvent_position   = cartesian_to_spherical(solvent_COM[nearest_15_solv_index])

    gF_solvent_orienation = compute_solvent_orientation(cwPOS,metadata,solvent_COM,nearest_15_solv_index)

    cwrPOS,metadata["RotationMatrix"] = align_solute(cwPOS,0.5*(cwPOS[16]+cwPOS[18]),cwPOS[7])
    _ = generate_POS_for_nearest(cwrPOS,metadata,15,AtomNames)

    #Features for forces
    #Transform forces using computing rotation matrix (to be with respect to local coordinates)

    rFRC = np.matmul(metadata["RotationMatrix"],np.transpose(FRC)).transpose()

    gF_force_per_solvent = compute_netForce_perSolvent(rFRC,metadata)
    gF_force_per_solute_atom = rFRC[:metadata["n_atoms_solute"],:]
    
    #Summarize shape of each feature to reference when normalizing data
    #feature_shapes = [np.shape(gF_solvent_distance),np.shape(gF_solvent_position),
    #                  np.shape(gF_solvent_orienation),np.shape(gF_force_per_solvent),
    #                  np.shape(gF_force_per_solute_atom)]
    #gF_concat = np.concatenate((gF_solvent_distance,gF_solvent_position,gF_solvent_orienation,
    #                            gF_force_per_solvent,gF_force_per_solute_atom),axis=None)
    
    #ORIGINAL FEATURES
    feature_shapes = [np.shape(gF_solvent_position),np.shape(gF_solvent_orienation),
                      np.shape(gF_force_per_solvent),np.shape(gF_force_per_solute_atom)]

    gF_concat = np.concatenate((gF_solvent_position,gF_solvent_orienation,
                               gF_force_per_solvent,gF_force_per_solute_atom),axis=None)
    #END ORIGINAL FEATURES
    
    #TEST FEATURES
    #feature_shapes = [np.shape(gF_solvent_position),np.shape(gF_force_per_solute_atom)]

    #gF_concat = np.concatenate((gF_solvent_position,gF_force_per_solute_atom),axis=None)
    #END TEST FEATURES
    return gF_concat, feature_shapes


In [37]:
def main():
    
    num_data=499
    #num_data=2
    feature_normalization = {}
    
    #Load the Vertical Energy Gaps for the System. These will be used as the reference to train the ML model.
    test_y = np.loadtxt("DATA/AQ_AQ-_VEE.dat").astype(np.float32)
    test_y = np.reshape(test_y,(len(test_y),1))
    
    #Initialize the arrays and lists used to temporarily store and construct the list of features
    featurearray=None
    featurelist = []
    for i in range(num_data):
        temp_array, temp_feature_shapes = load_and_analyze_trajstep("DATA/POS/POS_frame_"+str(i),"DATA/FRC/FRC_frame_"+str(i))
        featurelist.append(temp_array)
    featurearray = np.asarray(featurelist)
    
    def unison_shuffled_copies(a, b):
        assert len(a) == len(b)
        p = np.random.permutation(len(a))
        return a[p], b[p]
    
    #To avoid training the ANN to sequential data, we eliminate possible time-dependence by shuffling the data.
    #features and corresponding VEE gaps must be shuffled identically to correspond to one another during the
    #training process.
    featurearray,test_y = unison_shuffled_copies(featurearray,test_y)
       
    #Define split of data (80% train: 20% validation)
    train_size = int(0.8*np.shape(featurearray)[0])
    
    #Define training and validation sets
    X_train = featurearray[:train_size]
    y_train = test_y[:train_size]
    X_val = featurearray[train_size:]
    y_val = test_y[train_size:num_data]
    
    #Normalized Training and Validation Arrays
    nX_train = np.zeros_like(X_train)
    nX_val   = np.zeros_like(X_val)
    
    #NORMALIZE EACH GENERATED FEATURE (gF) SUCH THAT MEAN = 0 AND VARIANCE = 1
    #Normalization should only be computed using X_train set. The mean and variance should
    #then be stored and passed in order to shift the X_val and future X_test data 
    #appropriately to feed into the model.
    
    advance=0;
    for i in range(len(temp_feature_shapes)):
        num_feat=temp_feature_shapes[i][0] ; dim=temp_feature_shapes[i][1];
        temp_recon = X_train[:,advance:advance+(num_feat*dim)].reshape(len(X_train),num_feat,dim)
        temp_recon = temp_recon.reshape(len(X_train)*temp_feature_shapes[i][0],temp_feature_shapes[i][1])

        #Convert Polar to Cartesian, Compute Distance (r in polar), 
        if(i==0):
            #print(X_train[:,advance:advance+(num_feat*dim)])
            #print(temp_recon)
            solve_cart = spherical_to_cartesian(temp_recon)
            solvent_distances = np.linalg.norm(solve_cart,axis=1).reshape(len(temp_recon),1)
    
            ##Min-Max Normalization
            solvent_distances_min = np.min(solvent_distances)
            solvent_distances_max = np.max(solvent_distances)
            std_solvent_minmaxNorm = (solvent_distances-solvent_distances_min+1E-16)/(solvent_distances_max-solvent_distances_min)
            nSolvent_cart = std_solvent_minmaxNorm*solve_cart*(1.0/solvent_distances)
            nSolvent_spherical = cartesian_to_spherical(nSolvent_cart)
            nX_train[:,advance:advance+(num_feat*dim)] = nSolvent_spherical.reshape(len(X_train),num_feat*dim)
            
            #Update Feature Normalization Dictionary
            feature_normalization["gF_solvent_position_min"] = solvent_distances_min
            feature_normalization["gF_solvent_position_max"] = solvent_distances_max

        if(i==1):
            temp_min = np.min(temp_recon)
            temp_max = np.max(temp_recon)
            temp_minmaxNorm = (temp_recon-temp_min+1E-16)/(temp_max-temp_min)
            nX_train[:,advance:advance+(num_feat*dim)] = temp_minmaxNorm.reshape(len(X_train),num_feat*dim)
            
            #Update Feature Normalization Dictionary
            feature_normalization["gF_solvent_orienation_min"] = temp_min
            feature_normalization["gF_solvent_orienation_max"] = temp_max
        
        if(i==2 or i==3):
            force_mag = np.linalg.norm(temp_recon,axis=1).reshape(len(temp_recon),1)
            force_mean = np.mean(force_mag)
            force_std = np.std(force_mag)
            force_zScoreNorm = (force_mag-force_mean)/(force_std)
            force_zScoreNorm = force_zScoreNorm*(1.0/force_mag)*temp_recon
            nX_train[:,advance:advance+(num_feat*dim)] = force_zScoreNorm.reshape(len(X_train),num_feat*dim)
            
            if(i==2):
                #Update Feature Normalization Dictionary
                feature_normalization["gF_force_per_solvent_mean"] = force_mean
                feature_normalization["gF_force_per_solvent_std"] = force_std
            
            if(i==3):
                #Update Feature Normalization Dictionary
                feature_normalization["gF_force_per_solute_atom_mean"] = force_mean
                feature_normalization["gF_force_per_solute_atom_std"] = force_std
                
        advance+=(num_feat*dim)

    advance = 0
    
    for i in range(len(temp_feature_shapes)):
        num_feat=temp_feature_shapes[i][0] ; dim=temp_feature_shapes[i][1];
        temp_recon = X_val[:,advance:advance+(num_feat*dim)].reshape(len(X_val),num_feat,dim)
        temp_recon = temp_recon.reshape(len(X_val)*temp_feature_shapes[i][0],temp_feature_shapes[i][1])
        
        if(i==0):
            solve_cart = spherical_to_cartesian(temp_recon)
            solvent_distances = np.linalg.norm(solve_cart,axis=1).reshape(len(temp_recon),1)
    
            ##Min-Max Normalization
            solvent_distances_min = feature_normalization["gF_solvent_position_min"]
            solvent_distances_max = feature_normalization["gF_solvent_position_max"]
            std_solvent_minmaxNorm = (solvent_distances-solvent_distances_min+1E-16)/(solvent_distances_max-solvent_distances_min)
            nSolvent_cart = std_solvent_minmaxNorm*solve_cart*(1.0/solvent_distances)
            nSolvent_spherical = cartesian_to_spherical(nSolvent_cart)
            nX_val[:,advance:advance+(num_feat*dim)] = nSolvent_spherical.reshape(len(X_val),num_feat*dim)
        
        if(i==1):
            temp_min = feature_normalization["gF_solvent_orienation_min"]
            temp_max = feature_normalization["gF_solvent_orienation_max"]
            temp_minmaxNorm = (temp_recon-temp_min+1E-16)/(temp_max-temp_min)
            nX_val[:,advance:advance+(num_feat*dim)] = temp_minmaxNorm.reshape(len(X_val),num_feat*dim)
            
        if(i==2):
            force_mag = np.linalg.norm(temp_recon,axis=1).reshape(len(temp_recon),1)
            force_mean = feature_normalization["gF_force_per_solvent_mean"]
            force_std = feature_normalization["gF_force_per_solvent_std"]
            force_zScoreNorm = (force_mag-force_mean)/(force_std)
            force_zScoreNorm = force_zScoreNorm*(1.0/force_mag)*temp_recon
            nX_val[:,advance:advance+(num_feat*dim)] = force_zScoreNorm.reshape(len(X_val),num_feat*dim)
            
        if(i==3):
            force_mag = np.linalg.norm(temp_recon,axis=1).reshape(len(temp_recon),1)
            force_mean = feature_normalization["gF_force_per_solute_atom_mean"]
            force_std = feature_normalization["gF_force_per_solute_atom_std"]
            force_zScoreNorm = (force_mag-force_mean)/(force_std)
            force_zScoreNorm = force_zScoreNorm*(1.0/force_mag)*temp_recon
            nX_val[:,advance:advance+(num_feat*dim)] = force_zScoreNorm.reshape(len(X_val),num_feat*dim)
        
        advance+=(num_feat*dim)
        
    
    
    np.save("DATA/X_train",nX_train)
    np.save("DATA/X_val",nX_val)
    np.save("DATA/y_train",y_train)
    np.save("DATA/y_val",y_val)
    np.save("DATA/feature_normalization",feature_normalization)
        
    #return feature_normalization


In [38]:
def generate_Normalized_test_features(TEST_POS_DIR,TEST_FRC_DIR,num_data):
    
    #Load normalization constants generated using training data.
    feature_normalization = np.load("DATA/feature_normalization.npy",allow_pickle='TRUE').item()
    
    #Initialize the arrays and lists necessary to 
    featurearray=None
    featurelist = []
    for i in range(num_data):
        temp_array, temp_feature_shapes = load_and_analyze_trajstep("DATA/TEST/"+str(TEST_POS_DIR)+"/POS_frame_"+str(i),"DATA/TEST/"+str(TEST_FRC_DIR)+"/FRC_frame_"+str(i))
        featurelist.append(temp_array)
    
    
    X_test = np.asarray(featurelist)
    #Normalized Training and Validation Arrays
    nX_test = np.zeros_like(X_test)
    
    advance = 0
    
    #for i in range(len(temp_feature_shapes)):
    for i in range(4):
        num_feat=temp_feature_shapes[i][0] ; dim=temp_feature_shapes[i][1];
        temp_recon = X_test[:,advance:advance+(num_feat*dim)].reshape(len(X_test),num_feat,dim)
        temp_recon = temp_recon.reshape(len(X_test)*temp_feature_shapes[i][0],temp_feature_shapes[i][1])
        
        if(i==0):
            solve_cart = spherical_to_cartesian(temp_recon)
            solvent_distances = np.linalg.norm(solve_cart,axis=1).reshape(len(temp_recon),1)
    
            ##Min-Max Normalization
            solvent_distances_min = feature_normalization["gF_solvent_position_min"]
            solvent_distances_max = feature_normalization["gF_solvent_position_max"]
            std_solvent_minmaxNorm = (solvent_distances-solvent_distances_min+1E-16)/(solvent_distances_max-solvent_distances_min)
            nSolvent_cart = std_solvent_minmaxNorm*solve_cart*(1.0/solvent_distances)
            nSolvent_spherical = cartesian_to_spherical(nSolvent_cart)
            nX_test[:,advance:advance+(num_feat*dim)] = nSolvent_spherical.reshape(len(X_test),num_feat*dim)
        
        if(i==1):
            temp_min = feature_normalization["gF_solvent_orienation_min"]
            temp_max = feature_normalization["gF_solvent_orienation_max"]
            temp_minmaxNorm = (temp_recon-temp_min+1E-16)/(temp_max-temp_min)
            nX_test[:,advance:advance+(num_feat*dim)] = temp_minmaxNorm.reshape(len(X_test),num_feat*dim)

        if(i==2):
            force_mag = np.linalg.norm(temp_recon,axis=1).reshape(len(temp_recon),1)
            force_mean = feature_normalization["gF_force_per_solvent_mean"]
            force_std = feature_normalization["gF_force_per_solvent_std"]
            force_zScoreNorm = (force_mag-force_mean)/(force_std)
            force_zScoreNorm = force_zScoreNorm*(1.0/force_mag)*temp_recon
            nX_test[:,advance:advance+(num_feat*dim)] = force_zScoreNorm.reshape(len(X_test),num_feat*dim)
            
        if(i==3):
            force_mag = np.linalg.norm(temp_recon,axis=1).reshape(len(temp_recon),1)
            force_mean = feature_normalization["gF_force_per_solute_atom_mean"]
            force_std = feature_normalization["gF_force_per_solute_atom_std"]
            force_zScoreNorm = (force_mag-force_mean)/(force_std)
            force_zScoreNorm = force_zScoreNorm*(1.0/force_mag)*temp_recon
            nX_test[:,advance:advance+(num_feat*dim)] = force_zScoreNorm.reshape(len(X_test),num_feat*dim)
        advance+=(num_feat*dim)
        
    
    np.save("DATA/TEST/X_test",nX_test)
    

In [41]:
if __name__ == '__main__':
    

    ### STEP 1 ###
    """    
    This first step is step performed to normalize the features before feeding them into the ANN. As a result,
    the normalization parameters are saved. These parameters must be used for future data to apply to future
    data (i.e. loaded in generate_Normalized_test_features() ).
    """
    #main()

    ### STEP 2 ###
    """ 
    This step is used to properly scale and shift test data to be consistent with the training data.
    """
    generate_Normalized_test_features("POS_DFT","FRC_DFT",500)


## TEST CELLS BELOW

In [ ]:
[(15, 3), (15, 1), (95, 3), (24, 3)]
[(15, 3), (15, 1), (95, 3), (24, 3)]

In [837]:
npX_train = np.load("DATA/X_train.npy").astype(np.float32)

print(npX_train[0,:])

[ 9.66118872e-02  1.80178320e+00 -2.96182704e+00  2.51345992e-01
  1.43509400e+00  2.84603000e-01  4.10471946e-01  6.47507846e-01
 -2.65584779e+00  5.10221422e-01  1.73429406e+00  2.03423238e+00
  5.69994450e-01  2.07993746e+00  8.50595295e-01  6.00226462e-01
  2.24841380e+00 -5.86964250e-01  6.12429440e-01  2.39230227e+00
  2.72550702e+00  6.29202724e-01  8.40357006e-01 -7.24645197e-01
  6.62388563e-01  1.45837998e+00  1.25310934e+00  7.08181083e-01
  1.65139210e+00 -9.14409935e-01  7.15275407e-01  1.14977765e+00
  2.50871778e+00  8.32504988e-01  1.77373052e+00 -1.97482431e+00
  8.36285412e-01  8.20740104e-01  6.71886504e-01  8.63745391e-01
  7.49065697e-01 -1.80337882e+00  9.83092546e-01  2.50564122e+00
  1.64184654e+00  3.08519542e-01  4.88743931e-01  5.50470591e-01
  5.59637547e-01  1.64572284e-01  6.62935078e-01  6.01463974e-01
  7.08218575e-01  6.35603666e-01  6.52125061e-01  3.09627980e-01
  4.63429600e-01  8.42445135e-01  5.39400995e-01  2.98426986e-01
 -3.82229537e-01  1.18730

In [550]:
i=1
print(np.shape(featurearray))
print(np.shape(featurearray[:,advance:advance+(temp_feature_shapes[i][0]*temp_feature_shapes[i][1])].reshape((np.shape(featurearray)[0],-1,temp_feature_shapes[i][1])).T))

(499, 432)
(3, 15, 499)


In [553]:
featurearray[:,advance:advance+(temp_feature_shapes[i][0]*temp_feature_shapes[i][1])].reshape((np.shape(featurearray)[0],-1,temp_feature_shapes[i][1])).T[0,:,0]

array([3.8194768 , 5.54294444, 5.9688347 , 6.36782186, 6.90161912,
       3.8194768 , 4.46423767, 5.12729886, 5.54294444, 5.79201256,
       5.91798603, 5.9688347 , 6.03872732, 6.17700918, 6.36782186])

In [ ]:
#Define new X axis
newX = 0.5*(cwPOS[16]+cwPOS[18])
newX = (1.0/np.linalg.norm(newX))*newX
#Use a second point to define the plane.
#I.e. find a vector n orthogonal to a that is in the plane which contains a and b.
refY = cwPOS[7]
newY = np.cross(newX,np.cross(newX,(1.0/np.linalg.norm(refY))*refY))
newY = (1.0/np.linalg.norm(newY))*newY

#Define new Z as perpendicular to plane defined by newX and newY
newZ = np.cross(newX,newY)
newZ = (1.0/np.linalg.norm(newZ)*newZ)

#Apply reoriented molecules
alpha,beta,gamma = reorient_molecule(cwPOS,newX,newY,newZ)

#Define Rotation Matrix (Z1X2Z3, Euler Angles)

RotMatrix = np.array([[np.cos(alpha)*np.cos(gamma)-np.cos(beta)*np.sin(alpha)*np.sin(gamma),-np.cos(alpha)*np.sin(gamma)-np.cos(beta)*np.cos(gamma)*np.sin(alpha),np.sin(alpha)*np.sin(beta)],
                      [np.cos(gamma)*np.sin(alpha)+np.cos(alpha)*np.cos(beta)*np.sin(gamma),np.cos(alpha)*np.cos(beta)*np.cos(gamma)-np.sin(alpha)*np.sin(gamma),-np.cos(alpha)*np.sin(beta)],
                      [np.sin(beta)*np.sin(gamma),np.cos(gamma)*np.sin(beta),np.cos(beta)]])

test_rotation = np.matmul(RotMatrix,np.transpose(cwPOS)).transpose()


#test_roated_and_wrapped = wrap_cell(test_rotation,metadata,rot_translation_vectors)
#recon_rot_wrap,_,_ = generate_solvent_COMs(test_roated_and_wrapped,metadata)


In [ ]:
translation_vectors = np.array([[metadata["box_length"],0.0,0.0],[0.0,metadata["box_length"],0.0],[metadata["box_length"],0.0,0.0]])
rot_translation_vectors = np.matmul(RotMatrix,np.transpose(translation_vectors)).transpose()

rot_translation_vectors

#print(np.dot(a1,a2))
#print(np.linalg.norm(a1))
#cosine_angle = np.dot(a1, a2) / (np.linalg.norm(a1) * np.linalg.norm(a2))
#solvent_angles[i] = np.arccos(cosine_angle)

In [523]:
#print(dihedrals_and_centers,gF_dihedrals)
u1,u2,u3,u4 = cwPOS[dihedrals_and_centers[0]]
#print(u1,u2,u3,u4,gF_dihedrals)
#dihedrals_and_centers[2]

In [524]:
topology = metadata["topology_solute"]

dihedrals_and_centers = []

for pair1 in range(len(topology)-2):
    for pair2 in range(pair1+1,len(topology)-1):
        for pair3 in range(pair2+1,len(topology)):
            result_list = list(topology[pair1])
            result_list.extend(x for x in topology[pair2] if x not in result_list )
            result_list.extend(x for x in topology[pair3] if x not in result_list[2:] )
            if(len(result_list)==4):
                dihedrals_and_centers.append(result_list)
                
for four_atoms in dihedrals_and_centers:
    for i in topology:
        if [four_atoms[0],four_atoms[2]] == i:
            four_atoms[0],four_atoms[1] = four_atoms[1], four_atoms[0]

#dihedrals_and_centers

#    if set(tuple(x) for x in topology_reverse).intersection(four_atoms[1:3]):
#        #print("switch",four_atoms,four_atoms[1:3],pair)
#        
#        print(four_atoms,list(set(four_atoms[1:3]).intersection(pair)),pair)
    



In [525]:
solute_and_nearest = np.zeros(((metadata["n_atoms_solute"]+len(metadata["solvent_indices"])*15),3))
solute_and_nearest[:metadata["n_atoms_solute"],:] = cwPOS[:metadata["n_atoms_solute"],:]
for i, solv_index in enumerate(nearest_15_solv_index):
    solute_and_nearest[metadata["n_atoms_solute"]+i*len(metadata["solvent_indices"]):
                       metadata["n_atoms_solute"]+(i+1)*len(metadata["solvent_indices"]),:] = cwPOS[metadata["n_atoms_solute"]+solv_index*len(metadata["solvent_indices"]):
                       metadata["n_atoms_solute"]+(solv_index+1)*len(metadata["solvent_indices"]),:]

solute_and_nearest = np.matmul(RotMatrix,np.transpose(solute_and_nearest)).transpose()
write_xyz("AQ-_15nearestACN.xyz", solute_and_nearest, title="", atomtypes=AtomNames)

In [526]:
write_xyz('cwPOS_reconSolv_test.xyz', h, title="", atomtypes=AtomNames)
write_xyz('solventCOMs_test.xyz', solvent_COM, title="", atomtypes=95*["X"])

write_xyz("cPOS.xyz", cPOS, title="", atomtypes=AtomNames)
write_xyz("cwPOS.xyz", cwPOS, title="", atomtypes=AtomNames)

write_xyz("test_roated_and_wrapped.xyz", test_roated_and_wrapped, title="", atomtypes=AtomNames)
write_xyz("recon_rot_wrap.xyz", recon_rot_wrap, title="", atomtypes=AtomNames)

In [527]:
atomtypes=95*["X"]

In [833]:
print(temp_feature_shapes)

[(15, 1), (15, 3), (15, 1), (95, 3), (24, 3)]
